In [72]:
import pandas as pd
import numpy as np

In [73]:
data=pd.read_csv("train.csv")
df=data.drop(columns=["id"])

sample = np.random.choice(data.index, size=20000)
df = df.iloc[sample]

y=df['target']
y=y.values.reshape(20000,1)
# x=sample_df.drop('target', axis=1)
for j in range(20000):
    i=y[j]
    if i=='Class_1':
        y[j]=0
    if i=='Class_2':
        y[j]=1
    if i=='Class_3':
        y[j]=2
    if i=='Class_4':
        y[j]=3
    if i=='Class_5':
        y[j]=4
    if i=='Class_6':
        y[j]=5
    if i=='Class_7':
        y[j]=6
    if i=='Class_8':
        y[j]=7
    if i=='Class_9':
        y[j]=8

df=df.astype(float)
x=np.array(df.iloc[:,0:75])
x=np.divide(x,np.max(x,axis=0))

In [77]:
def sigmoid(x):
    return(np.divide(1,(1+np.exp(-x))))

def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))

# Output (prediction) formula
def output_formula(x, weights, bias):
    return(sigmoid(np.dot(x,weights)+bias))

# Error (log-loss) formula
def error_formula(y, output):
    return(-(np.dot(y.T,np.log(output))+np.dot((1-y).T,np.log(1-output))))

def error_term_formula(x, y, output):
    return (y-output)*np.dot(sigmoid_prime(x),x.T)

# Gradient descent step
def update_weights(x, y, weights, bias, learnrate):
    weights=weights + learnrate*((y-output_formula(x,weights,bias))*x)
    bias=bias+learnrate*(y-output_formula(x,weights,bias))
    return weights,bias


In [79]:
np.random.seed(44)
epochs = 100
learnrate = 0.001

def train(features,classes, epochs, learnrate):
    final_weights=[]
    for ind in range(0,9):
        targets= classes==ind
        targets=targets.astype(int)

        errors = []
        n_records, n_features = features.shape
        last_loss = None
        weights = np.random.normal(scale=1 / n_features**.5, size=n_features)
        bias = 0
        for e in range(epochs):
            del_w = np.zeros(weights.shape)
            for x, y in zip(features, targets):
                output=sigmoid(np.dot(x,weights))
                error_term=error_term_formula(x,y,output)
                del_w+=error_term
            weights+= learnrate*del_w
            out = sigmoid(np.dot(features, weights))
            loss = np.mean(error_formula(targets, out))
            if e % (epochs / 10) == 0:
                print("\n========== Epoch", e,"==========> Train loss: ", loss)
        final_weights.append(weights)
    return(final_weights)

final_weights=train(x,y,epochs,learnrate)


========== Epoch 0 ==========> Train loss:  9338.982043692173

========== Epoch 10 ==========> Train loss:  5978.095122298573

========== Epoch 20 ==========> Train loss:  5950.863515555184

========== Epoch 30 ==========> Train loss:  5950.8441032264545

========== Epoch 40 ==========> Train loss:  5950.8440626771735

========== Epoch 50 ==========> Train loss:  5950.844062591923

========== Epoch 60 ==========> Train loss:  5950.844062591743

========== Epoch 70 ==========> Train loss:  5950.844062591743

========== Epoch 80 ==========> Train loss:  5950.844062591743

========== Epoch 90 ==========> Train loss:  5950.844062591743

========== Epoch 0 ==========> Train loss:  13806.798416047977

========== Epoch 10 ==========> Train loss:  7553.919667040904

========== Epoch 20 ==========> Train loss:  7553.918553493406

========== Epoch 30 ==========> Train loss:  7553.918553493405

========== Epoch 40 ==========> Train loss:  7553.918553493405

========== Epoch 50 ==========> Train 

In [91]:
def cal_accuracy(features,classes,final_weights):
    count=0
    pridiction=np.zeros(len(classes))
    for x,y in zip(features,classes):
        final_output=[]
        for ind in range(0,9):
            weight=final_weights[ind]
            bias=0
            out = output_formula(x, weight, bias)
            final_output.append(out)
        label=np.argmax(final_output)
        if label==y:
            count+=1
    #print(count)
    return(count*100/len(classes))

acc=cal_accuracy(x,y,final_weights)
print(acc)

25.865


In [94]:
df1=pd.read_csv("test.csv")
dfy=df1["id"]
dfx=df1.drop(columns=["id"])
dfx = dfx.astype(float)

dfy=dfy.values.reshape(len(df1),1)
dfy=np.array(dfy)

dfx=np.divide(dfx,np.max(dfx,axis=0))
dfx=np.array(dfx)
df1=np.concatenate((dfy,dfx),axis=1)

In [103]:
pridiction=pd.DataFrame(columns=["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"])
ids=pd.DataFrame()

for x,y in zip(dfx,dfy):
        final_output=[]
        for ind in range(0,9):
            weight=final_weights[ind]
            bias=0
            out = output_formula(x, weight, bias)
            final_output.append(out)
            #print(final_output)
        a_series = pd.Series(final_output, index = pridiction.columns)
        pridiction = pridiction.append(a_series, ignore_index=True)
        #pridict=pd.DataFrame(final_output,columns=["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"])
        #pridiction.iloc[length]=final_output


In [115]:
pridiction=np.array(pridiction)
pridiction=pridiction/pridiction.sum(axis=1, keepdims=True)
pridiction=pd.DataFrame(pridiction,columns=["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"])

In [116]:

dfy=dfy.astype(int)
dfy=pd.DataFrame(dfy,columns=["id"])
df=pd.concat([dfy,pridiction],axis=1)
df.to_csv('final_pridictions_linear_reg.csv')